## PyCity Schools Analysis
#### - The amount of money the school spends per student is inversely proportional to their performance in math and reading tests.
#### - Charter school students do much better than district school students.
#### - Schools with less than 2000 students do much better than schools with more than 2000 students.

In [69]:
import pandas as pd
import numpy as np

df_students = pd.read_csv("students_complete.csv")
df_schools = pd.read_csv("schools_complete.csv")

# total math and reading passing grade added to school df
ayy = pd.DataFrame(df_students['school'].loc[df_students['math_score'] >= 70].value_counts())
lmao = pd.DataFrame(df_students['school'].loc[df_students['reading_score'] >= 70].value_counts())
ayy.reset_index(inplace=True)
lmao.reset_index(inplace=True)
ayylmao = pd.merge(ayy,lmao,on='index')
ayylmao.columns = ["name", "math_pass","reading_pass"]
ayylmao
df_schools = pd.merge(df_schools,ayylmao,on='name')

### Distric Summary

In [70]:
math_path = df_students["math_score"].loc[df_students["math_score"] >= 70].count()/df_schools['size'].sum() * 100
read_path = df_students["reading_score"].loc[df_students["reading_score"] >= 70].count()/df_schools['size'].sum() * 100
dist_sum = pd.DataFrame(
        {"Total Schools": df_schools['name'].count(),
         "Total Students": "{:,}".format(df_schools['size'].sum()),
         "Total Budget": "${:,.2f}".format(df_schools['budget'].sum()),
         "Average Math Score": "{:.2f}".format(df_students['math_score'].mean()),
         "Average Reading Score": "{:.2f}".format(df_students ['reading_score'].mean()),
         "% Passing Math": "{:.2f}%".format(math_path),
         "% Passing Reading": "{:.2f}%".format(read_path),
         "% Overall Passing Rate": "{:.2f}%".format((math_path+read_path)/2)}, index = [0])
dist_sum = dist_sum[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                     "Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
dist_sum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


### School Summary

In [71]:
sch_sum = df_schools.loc[:,['name','type','size','budget']]
sch_sum.columns = ['name','School Type','Total Students','Total School Budget']
sch_sum['Per Student Budget'] = df_schools['budget']/df_schools['size']
sch_sum['% Passing Math'] = df_schools['math_pass']/df_schools['size']*100
sch_sum['% Passing Reading'] = df_schools['reading_pass']/df_schools['size']*100
sch_sum['% Overall Passing Rate'] = (sch_sum['% Passing Math']+sch_sum['% Passing Reading'])/2
sch_sum = sch_sum.set_index('name')
sch_sum.index.name = None
sch_sum.sort_index(inplace=True)
sch_sum['Average Math Score'] = df_students.groupby('school')['math_score'].mean()
sch_sum['Average Reading Score'] = df_students.groupby('school')['reading_score'].mean()
sch_cols = sch_sum.columns.tolist()
sch_cols = sch_cols[:4] + sch_cols[7:] + sch_cols[4:7]
sch_sum = sch_sum[sch_cols]
sch_sum_final = sch_sum.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget":"${:,.2f}", "Average Math Score":"{:.2f}", 
                      "Average Reading Score":"{:.2f}", "% Passing Math":"{:.2f}%", 
                      "% Passing Reading":"{:.2f}%", "% Overall Passing Rate":"{:.2f}%"})
sch_sum_final

## Top Performing Schools (By Passing Rate)

In [72]:
top_sch = sch_sum.sort_values(['% Overall Passing Rate'],ascending = False).head()
top_sch = top_sch.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget":"${:,.2f}", "Average Math Score":"{:.2f}", 
                      "Average Reading Score":"{:.2f}", "% Passing Math":"{:.2f}%", 
                      "% Passing Reading":"{:.2f}%", "% Overall Passing Rate":"{:.2f}%"})
top_sch

## Bottom Performing Schools (By Passing Rate)

In [73]:
bot_sch = sch_sum.sort_values(['% Overall Passing Rate'],ascending = True).head()
bot_sch = bot_sch.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget":"${:,.2f}", "Average Math Score":"{:.2f}", 
                      "Average Reading Score":"{:.2f}", "% Passing Math":"{:.2f}%", 
                      "% Passing Reading":"{:.2f}%", "% Overall Passing Rate":"{:.2f}%"})
bot_sch

## Math Scores by Grade

In [74]:
math_temp = df_students.loc[:,['grade','school','math_score']]
math_gr_9 = math_temp.loc[math_temp['grade'] == "9th"].groupby(['school']).mean()
math_gr_9.reset_index(level=0, inplace=True)
math_gr_10 = math_temp.loc[math_temp['grade'] == "10th"].groupby(['school']).mean()
math_gr_10.reset_index(level=0, inplace=True)
math_gr_11 = math_temp.loc[math_temp['grade'] == "11th"].groupby(['school']).mean()
math_gr_11.reset_index(level=0, inplace=True)
math_gr_12 = math_temp.loc[math_temp['grade'] == "12th"].groupby(['school']).mean()
math_gr_12.reset_index(level=0, inplace=True)
math_gr = math_gr_9.merge(math_gr_10,on = 'school').merge(math_gr_11,on = 'school').merge(math_gr_12,on='school')
math_gr.columns = ['school','9th','10th','11th','12th']
math_gr = math_gr.set_index('school')
del math_gr.index.name
math_gr = math_gr.style.format({"9th": "{:.2f}", "10th":"{:.2f}", "11th":"{:.2f}","12th":"{:.2f}"})
math_gr

## Reading Score by Grade

In [75]:
read_temp = df_students.loc[:,['grade','school','reading_score']]
read_gr_9 = read_temp.loc[read_temp['grade'] == "9th"].groupby(['school']).mean()
read_gr_9.reset_index(level=0, inplace=True)
read_gr_10 = read_temp.loc[read_temp['grade'] == "10th"].groupby(['school']).mean()
read_gr_10.reset_index(level=0, inplace=True)
read_gr_11 = read_temp.loc[read_temp['grade'] == "11th"].groupby(['school']).mean()
read_gr_11.reset_index(level=0, inplace=True)
read_gr_12 = read_temp.loc[read_temp['grade'] == "12th"].groupby(['school']).mean()
read_gr_12.reset_index(level=0, inplace=True)
read_gr = read_gr_9.merge(read_gr_10,on = 'school').merge(read_gr_11,on = 'school').merge(read_gr_12,on='school')
read_gr.columns = ['school','9th','10th','11th','12th']
read_gr = read_gr.set_index('school')
del read_gr.index.name
read_gr = read_gr.style.format({"9th": "{:.2f}", "10th":"{:.2f}", "11th":"{:.2f}","12th":"{:.2f}"})
read_gr

## Scores by School Spending

In [76]:
sss_temp = sch_sum.loc[:,['Per Student Budget','Average Math Score','Average Reading Score',
                          '% Passing Math','% Passing Reading']]
sss = sss_temp.groupby(['Per Student Budget'],as_index=False).mean()
sss1 = sss.loc[sss['Per Student Budget'] < 585].mean()
sss2 = sss.loc[(sss['Per Student Budget'] >= 585) & (sss['Per Student Budget'] < 615)].mean()
sss3 = sss.loc[(sss['Per Student Budget'] >= 615) & (sss['Per Student Budget'] < 645)].mean()
sss4 = sss.loc[(sss['Per Student Budget'] >= 645) & (sss['Per Student Budget'] < 675)].mean()
sss_new = pd.DataFrame([sss1,sss2,sss3,sss4])
sss_new['% Overall Passing Rate'] = (sss_new['% Passing Math'] + sss_new['% Passing Reading'])/2
del sss_new['Per Student Budget']
sss_new['Spending Ranges (Per Student)'] = ['<$585','$585-615','$615-645','$645-675']
sss_new.set_index(['Spending Ranges (Per Student)'],inplace = True)
sss_new = sss_new.style.format({"Average Math Score": "{:.2f}", "Average Reading Score":"{:.2f}", 
                                "% Passing Math":"{:.2f}%","% Passing Reading":"{:.2f}%",
                               "% Overall Passing Rate":"{:.2f}%"})
sss_new

## Scores by School Size

In [77]:
ssz_temp = sch_sum.loc[:,['Total Students','Average Math Score','Average Reading Score',
                          '% Passing Math','% Passing Reading']]
ssz = ssz_temp.groupby(['Total Students'],as_index=False).mean()
ssz1 = ssz.loc[ssz['Total Students'] < 1000].mean()
ssz2 = ssz.loc[(ssz['Total Students'] >= 1000) & (ssz['Total Students'] < 2000)].mean()
ssz3 = ssz.loc[(ssz['Total Students'] >= 2000) & (ssz['Total Students'] < 5000)].mean()
ssz_new = pd.DataFrame([ssz1,ssz2,ssz3])
ssz_new['% Overall Passing Rate'] = (ssz_new['% Passing Math']+ssz_new['% Passing Reading'])/2
del ssz_new['Total Students']
ssz_new['School Size'] = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']
ssz_new.set_index(['School Size'],inplace = True)
ssz_new = ssz_new.style.format({"Average Math Score": "{:.2f}", "Average Reading Score":"{:.2f}", 
                                "% Passing Math":"{:.2f}%","% Passing Reading":"{:.2f}%",
                               "% Overall Passing Rate":"{:.2f}%"})
ssz_new

## Scores by School Type

In [78]:
sst_temp = sch_sum.loc[:,['School Type','Average Math Score','Average Reading Score',
                         '% Passing Math','% Passing Reading']]
sst = sst_temp.groupby('School Type')['Average Math Score', "Average Reading Score","% Passing Math",
                               '% Passing Reading'].mean()
sst['% Overall Passing Rate'] = (sst['% Passing Math']+sst['% Passing Reading'])/2
sst_new = sst.style.format({'Average Math Score': "{:.2f}", "Average Reading Score":"{:.2f}", 
                        "% Passing Math":"{:.2f}%","% Reading Math":"{:.2f}%","% Overall Passing Rate":"{:.2f}%"})
sst_new